# Function Design Guideline
- Use arguments for inputs and  return for outputs
    - Make a function independent of things outside of it
- Avoid using global variable and mutable arguments
- Each function should have a single, unified purpose
- Each function should be small
- **Minimize external dependencies in functions and other program components** -> self-containied

---

# Recursive Functions
Recursion is not used as often in Python as in language like Prolog or Lisp, because Python emphasizes simpler procedural statements like loops.  
However, this technique can still be usefull.  

- The misuse of the following operator overloadings might cause inproper recursive behavior
    - `__setattr__`, `__getattribute__`, `__repr__`

Standard Python limits the depth of its runtime call stack to trap infinite recursion.

In [1]:
# To see the recursion limit
import sys
sys.getrecursionlimit()

1000

In [2]:
# To set the recursion limit
sys.setrecursionlimit(10000)
sys.getrecursionlimit()

10000

---

# Function Objects

**First-Class Object Model**: Functions are just a kind of object.
- Can be assigned to other names
- Can be passed as arguement
- Can be embedded in data strcutures
- Can be returned and other operations that other object can do.  

This first-class object model and lacking of type declarations make Python incredibly flexible.

## Function Attributes
It's is possible to attach user-defined attributes to functions

In [3]:
def func():
    print('hi')

func.text = 'text'
print(func.text)

text


In [4]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'text']

Such attributes can be used to attach **state information** to function object.  
This way can be used to emulate "static locals" in other languages - variables whose names are local to a function

## Function Annotations (Python3)
Attach values to function attribute `__annotations__` for use by other tools.  
Annotations work only in **def**, not **lambda

In [5]:
def func(a: 1, b: 'a', c: int) -> int:
    return a + b + c

print(func(1, 2, 3))

6


In fact, calls to an annotated function work as usual.

In [6]:
# Call __annotations__

func.__annotations__

{'a': 1, 'b': 'a', 'c': int, 'return': int}

### Annotation with default

In [7]:
def func(a: 1 = 4, b: '1' = 5, c: int = 6) -> int:
    return a + b + c

print(func())

15


The potential uses remain to be uncovered.  
Larger APIs might use this feature as a way to register function interface information.

---

# Anonymous Functions: lambda

## Usage
- Inline a function definition
- Defer execution of a piece of code

```python
lambda arg1, arg2, ... argN : expression using args
```

- `lambda` is an expression, not a statement
    - It can apper in place that `def` cannot. Such as inside a list or a function call's argument
- `lambda`'s body is a single expression
    - It's designed for simple functions, and `def` handles larger tasks.

`lambda` also follows the LEGB rule described in CH17

In [8]:
f = lambda x, y: x + y
f(1, 2)

3

## Why Use lambda
- Common usage
    - Callback handler
    - Jump table: lists or dicts of actions to be performed on demand
    - Short function
    - Inline temporary function definitions not used elsewhere

In [9]:
# Multiway branch switches

s = {'a': (lambda: 1 + 1),
     'b': (lambda: 2 + 2)}

s['a']()

2

In [10]:
# lambda with if

less = lambda x, y: x if x < y else y
less(2, 1)

1

In [11]:
# lambda with loop (map or list comprehension)

print_all = lambda x: [print("This is content: " + line) for line in x]
print_all(('a', 'b', 'c'))

This is content: a
This is content: b
This is content: c


[None, None, None]

Assignments cannot be directly achieve through ``lambda`

---

# FP tools - map, filter, reduce


## map
Passing each item to the function and collects all results (Returning a list in Python2, an iterator in Python3)

map applies a function call to each item insteasd of expression, and thus often requires extra helper functions or lambdas.

In [12]:
list(map(lambda x: x + 3, [1, 2, 3]))

[4, 5, 6]

With multiple sequences, `map` expects an N-argument function for N sequences.

In [13]:
list(map(pow, [1, 2, 3], [4, 5, 6]))

[1, 32, 729]

In some case, **map** may be faster to run than a list comprehension (e.g. built-in functions)  
But in other cases, it often slower.

## filter
Collecting items for which the function returns a True value

In [14]:
list(filter(lambda x: x > 0, range(-5, 5)))

[1, 2, 3, 4]

## reduce
Computing a single value by applying the function to an accumulator and successive items.

- It's built-in in Python2, but in `functools` module in Python3.
- **operator** provide functions that correspond to built-in expressions 

In [15]:
import operator, functools
functools.reduce(operator.add, [2, 4, 6])

12

In [16]:
# Demonstrate how reduce really works

def add(a, b):
    print("Current Result: {}\tNext value to be added: {}".format(a, b))
    print("After add: {}\n".format(a + b))
    return a + b

functools.reduce(add, [1, 2, 3, 4, 5])

Current Result: 1	Next value to be added: 2
After add: 3

Current Result: 3	Next value to be added: 3
After add: 6

Current Result: 6	Next value to be added: 4
After add: 10

Current Result: 10	Next value to be added: 5
After add: 15



15